Spreading Out

Heat equation in one spatial dimension:

$$\frac{\partial T}{\partial t} = \alpha \frac{\partial^2 T}{\partial x^2}$$

Implicit method works differently: we will use more data from the "future" in the update, including several values of T at $t^{n+1}$. This will make the scheme more difficut to apply.

Problems with explicit method:
1) boundary effects drag behind by one time step
2) stability requirements constrain the time step to very small values.

Both of those are resovled by implicit method!

Let's try combining the Euler time step with an evaluation of the spatial derivative on the updated solution at $t^{n+1}$

$$\frac{T_i^{n+1} - T_i^n}{\Delta t} = \alpha \frac{T_{i+1}^{n+1} - 2T_i^{n+1} + T_{i-1}^{n+1}}{\Delta x^2}$$

From the previous time step we only know $T_i^n$ but what about $T_{i+1}^{n+1}$, $T_{i-1}^{n+1}$ and $T_i^{n+1}$

If we put knowns on the RHS and unkowns on the LHS:

$$-T_{i-1}^{n+1} + \left( 2+\frac{\Delta x^2}{\alpha \Delta t} \right) T_i^{n+1} - T_{i+1}^{n+1} = T_i^n \frac{\Delta x^2}{\alpha \Delta t}$$

It looks like alot of unkowns and just one equation!

With i=1

$$-T_{0}^{n+1} + \left( 2+\frac{\Delta x^2}{\alpha \Delta t} \right) T_1^{n+1} - T_{2}^{n+1} = T_1^n \frac{\Delta x^2}{\alpha \Delta t}$$

and i = 2:

$$-T_{1}^{n+1} + \left( 2+\frac{\Delta x^2}{\alpha \Delta t} \right) T_2^{n+1} - T_{3}^{n+1} = T_2^n \frac{\Delta x^2}{\alpha \Delta t}$$

and i = 3:

$$-T_{2}^{n+1} + \left( 2+\frac{\Delta x^2}{\alpha \Delta t} \right) T_3^{n+1} - T_{4}^{n+1} = T_3^n \frac{\Delta x^2}{\alpha \Delta t}$$

You can see a common element across equations as the element moves from right to left. See Numerical mooc lesson for image!

$T_i^{n+1}$ also appears in the euation for $T_{i-1}^{n+1}$ and $T_{i+1}^{n+1}$ This is a linear system of equations for the unkown values $T_i^{n+1}$


What about the boundary conditions?

We have Dirichlet BC at x=0 and a Nuemann BC at x = 1.

The term $T_0^{n+1}$ is known at every time step from the BC so putting all unkowns on LHS and knowns on RHS yeilds for i = 1:

$$-T_{2}^{n+1} + \left( 2+\frac{\Delta x^2}{\alpha \Delta t} \right) T_1^{n+1} = T_1^n \frac{\Delta x^2}{\alpha \Delta t} + - T_{0}^{n+1}$$

On the other hand for i = N-2 the equation reads:

$$-T_{N-3}^{n+1} + \left( 2+\frac{\Delta x^2}{\alpha \Delta t} \right) T_{N-2}^{n+1} - T_{N-1}^{n+1} = T_{N-2}^n \frac{\Delta x^2}{\alpha \Delta t}$$


The discretized NBC on the RHS of the rod is:

$$\frac{T_{N-1}^n - T_{N-2}^n}{\Delta x} = q$$

But we can just as easily write that for the next time step:

$$\frac{T_{N-1}^{n+1} - T_{N-2}^{n+1}}{\Delta x} = q$$

Inserting the NBC in the equation for i = N-2 yields:

$$-T_{N-3}^{n+1} + \left( 1+\frac{\Delta x^2}{\alpha \Delta t} \right) T_{N-2}^{n+1} = T_{N-2}^n \frac{\Delta x^2}{\alpha \Delta t} + \Delta x q$$

Now we can write the linear system of equations out in matrix form as follows:

$$[A][x] = [b]+[b]_{b.c.}$$

Where the matrix coefficients [A] is a sparse matrix, most of the elements are zero with three nonzero diagonals. with $\sigma = \frac{\alpha \Delta t}{\Delta x^2}$:

\begin{align}\left[ \begin{array}{cccccc}
 \left(2 + \frac{1}{\sigma}\right) & -1 & 0 & \cdots & & 0 \\
 -1 & \left(2 + \frac{1}{\sigma}\right) & -1 & 0 & \cdots & 0 \\
 0 & & \ddots& & & \vdots \\
 \vdots & & & & \left(2 + \frac{1}{\sigma}\right)& \\
 0 & \cdots & & & -1 & \left(1 + \frac{1}{\sigma}\right) \end{array} \right]
 \cdot 
 \left[ \begin{array}{c} 
 T_1^{n+1} \\ T_2^{n+1} \\ \vdots \\ \\ T_{N-2}^{n+1} \end{array} \right]
 =
 \left[ \begin{array}{c} 
T_1^n \frac{1}{\sigma} \\ T_2^{n}\frac{1}{\sigma} \\ \vdots \\ \\ T_{N-2}^{n}\frac{1}{\sigma} \end{array} \right]
 +
 \begin{bmatrix}
  T_0^{n+1}\\
  0\\\\
  \vdots\\\\
  0\\
  q\Delta x
 \end{bmatrix}
 \end{align} 
 
 Notice the Dirichlet BC adds only a term to the RHS. The NBC adds both a term to the RHS and modifies the matrix A.
 
 

In [1]:
#Problem Set up:

import numpy
from matplotlib import pyplot
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

In [2]:
L = 1.
nt = 100
nx = 51
alpha = 1.22e-3

q = 0.

dx = L/(nx-1)

qdx = q*dx

Ti = numpy.zeros(nx)

Ti[0] = 100


In [3]:
#Solving a linear system

from scipy.linalg import solve

In [4]:
def generateMatrix(N,sigma):
    #setup diaganol
    d = numpy.diag(numpy.ones(N-2)*(2+1./sigma))
    
    #consider NBC
    d[-1,-1] = 1+1./sigma
    
    #setup upper diag
    ud = numpy.diag(numpy.ones(N-3)*-1,1)
    
    #setup lower diag
    ld = numpy.diag(numpy.ones(N-3)*-1,-1)
    
    A = d + ud + ld
    
    return A

In [5]:
def generateRHS(T,sigma,qdx):
    
    b = numpy.zeros_like(T)
    
    b = T[1:-1]*1./sigma
    
    #consider DBC
    b[0] += T[0]
    
    #consider NBC
    b[-2] += qdx
    
    return b


In [6]:
#Define function that steps in time

def implicit_ftcs(T,A,nt,sigma,qdx):
    
    for t in range(nt):
        Tn = T.copy()
        b = generateRHS(Tn, sigma,qdx)
        
        #Use numpy.linalg.solve
        T_interior = solve(A,b)
        T[1:-1] = T_interior
        
        #Enfore NBC, DBC is enforced automatically
        T[-1] = T[-2] + qdx
    return T